# Getting started

This section is here to help you get started with Skchange. It provides a range of examples to show the span of the things you can do with the library. For a more in-depth introduction to the core concepts of Skchange, please skip to the [Concepts](./concepts/index.rst) section.
 <!-- It covers the main features and basic usage of the library to solve change or anomaly detection problems. -->

## Installation
```bash
pip install skchange
```

To make full use of the library, you can install the optional Numba dependency. This will speed up the computation of the algorithms in Skchange, often by as much as 10-100 times.

```bash
pip install skchange[numba]
```

## Change detection

### Detect changes in the mean

In [ ]:
import plotly.io as pio

pio.renderers.default = "notebook"

In [ ]:
import plotly.express as px

from skchange.change_detectors import SeededBinarySegmentation
from skchange.change_scores import CUSUM
from skchange.datasets import generate_changing_data

x = generate_changing_data(
    n=200,
    change_points=[30, 35, 50, 100, 160],
    means=[0, 10, 0, -3, 5, 1],
    random_state=11,
)
detector = SeededBinarySegmentation(
    CUSUM(),
    penalty=5,
)
changepoints = detector.fit_predict(x)

fig = px.line(x)
for cpt in changepoints["ilocs"]:
    fig.add_vline(x=cpt, line_width=2, line_dash="dash", line_color="red")
fig.show()
changepoints

### Detect changes in a continuous piecewise linear trend

In [ ]:
from skchange.change_scores import ContinuousLinearTrendScore
from skchange.datasets import generate_continuous_piecewise_linear_data

x = generate_continuous_piecewise_linear_data(
    n=200,
    change_points=[30, 50, 100, 160],
    slopes=[0, 1, -0.5, 0.5, 0.1],
    random_state=2,
)
detector = SeededBinarySegmentation(
    ContinuousLinearTrendScore(),
    penalty=20,
    selection_method="narrowest",
)
changepoints = detector.fit_predict(x)

fig = px.line(x)
for cpt in changepoints["ilocs"]:
    fig.add_vline(x=cpt, line_width=2, line_dash="dash", line_color="red")
fig.show()
changepoints

### Detect changes in a linear regression model

In [ ]:
from skchange.change_detectors import PELT
from skchange.costs import LinearRegressionCost

### Detect changes in a high-dimensional mean vector

In [ ]:
from skchange.change_detectors import MovingWindow
from skchange.change_scores import ESACScore

## Segment anomaly detection

### Detect segment anomalies in the mean

In [ ]:
from skchange.anomaly_detectors import CAPA
from skchange.anomaly_scores import L2Saving

### Detect segment anomalies in the covariance matrix

In [ ]:
from skchange.anomaly_detectors import CircularBinarySegmentation
from skchange.costs import MultivariateGaussianCost

### Detect segment anomalies in multivariate data and identify the anomalous variables

In [ ]:
from skchange.compose.penalised_score import PenalisedScore
from skchange.datasets import generate_anomalous_data
from skchange.penalties import make_linear_chi2_penalty

n = 300
anomalies = [(100, 120), (250, 300)]
means = [[8.0, 0.0, 0.0], [2.0, 3.0, 5.0]]
df = generate_anomalous_data(n, anomalies, means, random_state=3)
p = df.shape[1]

score = L2Saving()
penalty = make_linear_chi2_penalty(score.get_model_size(1), n, p)
penalised_score = PenalisedScore(score, penalty)
detector = CAPA(penalised_score, find_affected_components=True)
detector.fit_predict(df)